In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Libraries and functions

In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
import time
import math
from tqdm import tqdm
from importlib import reload
from os.path import join, normpath
from bs4 import BeautifulSoup
import os
import roamability as rb

downloads = 'C:/Users/balob/Downloads'

#### To get the path

In [95]:
from pathlib import Path
path_dir = Path.cwd()
print(path_dir)
path_dir

C:\Users\balob\Documents\GITLAB\RB_BD


WindowsPath('C:/Users/balob/Documents/GITLAB/RB_BD')

In [94]:
pwd=!echo %cd%
pwd=str(pwd)
pwd=r"c:\Users\balob\Documents\GITLAB\NOC_Roamability_com\noc\noc"
pwd.replace('\\','/',100)

'c:/Users/balob/Documents/GITLAB/NOC_Roamability_com/noc/noc'

In [ ]:
%reset -f

# Examples

### Soap API OCS Portal

In [258]:
#%reset -f
from zeep import Client
from zeep.wsse.username import UsernameToken

connect = input("Choose connection (prod or lab):")

if connect == 'lab':
    user = 'lab_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD' #Lab
    #api_link = 'https://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
    api_link = 'https://labocsapi.roamability.com:6443/ocsapi/roamability/api/roamability.wsdl'
elif connect == 'prod':
    user = 'prod_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD'
    api_link = 'http://172.20.35.12:8585/ocsapi/roamability/api/roamability.wsdl'

user_name_token = UsernameToken(user, password)
user_name_token.use_digest = True
client = Client(api_link, wsse=user_name_token)

reseller = client.service.getResellerInfo()
accounts = client.service.getAccounts()

if reseller.result.code == '1':
    print('You are connected to Reseller: {}.\
    \nAvailable credit for the Reseller: {:.0f} USD.\n'.\
    format(reseller.reseller.resellerName,reseller.reseller.availableCredit))
else:
    print('Error in getting Reseller info: {}'.format(reseller.result.description))

if accounts.result.code == '1':
    accounts_dict = {account.accountId:account.accountName for account in accounts.accounts.accounts}
    print('The Reseller contains the following accounts:')
    for i,[account_id,account_name] in enumerate(accounts_dict.items()):
        print(' {:<5}{} (accountID: {})'.format(i,account_name,account_id))
else:
    print('Error in getting Account info: {}'.format(accounts.result.description))
    
# Subscriber in lab in STI account
#subscriber_id = client.service.getSubscriberById('934311')
#subscriber_iccid = client.service.getSubscriberByICCID('8997219121000031446')
#subscriber_id

Choose connection (prod or lab):lab
You are connected to Reseller: Porto Seguro DMI RT.    
Available credit for the Reseller: 2759 USD.

The Reseller contains the following accounts:
 0    Porto Seguro Conecta (accountID: 352327)
 1    Porto Seguro disconnected subscribers (accountID: 352434)


### Send MSU

In [423]:
from bs4 import BeautifulSoup
import roamability as rb
#reload(rb)

ogt = {'p4':'48790993070',
       'partner':'97254120624',
       'sure':'447797706411',
       'tot':'66893773228',
       'porto_seguro':'550549900000',
       'rusec':'79028710069',
       'multi_byte_sponsor':'85263347864'}

rb.greetings_func()
#rb.print_imsi_prof('48790993070', '250350013158002', '79028710099', 'as_resp') #'as_mo'
#soup = BeautifulSoup(rb.sri4sm(48790993070, 17708103467),'xml') #250359000007598 79028710043

#rb.print_imsi_prof(ogt['rusec'], '250350013158002', '79028710099', 'as_resp') #310380608103467/19703769255

#soup = BeautifulSoup(rb.sri4sm(ogt, msisdn),'xml')
#soup = BeautifulSoup(rb.prn(ogt, dgt, imsi),'xml')
#soup = BeautifulSoup(rb.sai(ogt, dgt, imsi, node),'xml') # 0 - vlr; 16 - mme
#soup = BeautifulSoup(rb.sendSMS(ogt, imsi, msc, rb.get_info),'xml')

#rb.print_imsi_prof(ogt['p4'], '260060140073744', '46705014099', 'as_resp') #310380608103467/19703769255
#soup = BeautifulSoup(rb.prn(ogt['p4'], '46705014099', '260060140073744'),'xml')

switch_01_payload = '24027000001F0D00010000BFFF0100000000000010F5375954EF29FB9A7F41ADE8444236A3'


#switch_01_payload = '24027000001f0d00010000bfff0100000000000010554f212dbbad49a1caf2573e1a626139'
switch_02_payload = '24027000001F0D00010000BFFF0100000000000040a740AD174703383798A7A6545BB36491'

soup = BeautifulSoup(rb.sendSMS(ogt['p4'], '260060140090204', '13123149088', switch_01_payload),'xml')
soup

Greetings from Roamability!!!


<?xml version="1.0" encoding="utf-8"?>
<response>
<result>0</result><response>00077FF61502710000100ABFFF0100000000000000FFFFFF8B00</response>
<src_gt>13123149086</src_gt>
</response>

 ### Format output

In [ ]:
import datetime as dt
print( 'start: {} {} :end'.format('one', 'two') ) #start: one two :end
print( 'start: {1} {0} :end'.format('one', 'two') ) #start: two one :end
print( 'start: test {:10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:10} :end'.format(10) ) #start: test         10 :end
print( 'start: test {:>10} :end'.format('test') ) #start: test       test :end
print( 'start: test {:<10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:_<10} :end'.format('test') ) #start: test test______ :end
print( 'start: test {:^10} :end'.format('test') ) #start: test    test    :end
print( 'start: test {:.10} :end'.format('test1test2test3') ) #start: test test1test2 :end
print( 'start: test {:_^10.5} :end'.format('test1test2test3') ) #start: test __test1___ :end
print( 'start: test {:d} :end'.format(42) ) #start: test 42 :end
print( 'start: test {:f} :end'.format(42) ) #start: test 42.000000 :end
print( 'start: test {:06.2f} :end'.format(3.141592) ) #start: test 003.14 :end
print( 'start: test {p[first]} {p[last]} :end'.format(p={'first': 'Jean-Luc', 'last': 'Picard'}) ) #start: test Jean-Luc Picard :end
print( 'start: test {p[4]} {p[5]} :end'.format(p=[4, 8, 15, 16, 23, 42]) ) #start: test 23 42 :end
print( 'start: test {:%Y-%m-%d %H:%M} :end'.format(dt.datetime(2001, 2, 3, 4, 5)) ) #start: test 2001-02-03 04:05 :end

### Search in files by Tags

In [14]:
import roamability as rb
reload(rb)

# Где искать файлы .txt с тегами
paths = ['c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_HOW_IT_WORKS/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/',
        'c:/W_DATA_ROAM/ПРОЕКТЫ/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_KNOWN_ISSUES/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/NOC_M_NEW_CONNECTION/',
        'c:/W_DATA_ROAM/ALARMS/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES',
        ]

# Если хотя бы один из тэгов присутствует. Нужно указать минимум один.
any_tags  = ['#ocs_principle_of_operation']

# Если все теги присутствуют. Нужно оставить лист пустым [] если нет обязательных тэгов.
all_tags = []

rb.find_files_by_tags(paths, all_tags, any_tags)

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\CAMEL_Relay.txt
----------------------------------------------------------------------------------------------------
****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\HTTP2SMPP.txt
----------------------------------------------------------------------------------------------------
#ocs_principle_of_operation
#sms_principle_of_operation
------------------------------
На OCSAPP реализован сервис http2smpp.
С его помощью можно отправлять SMS через http запросы.
На 25.12.2019 используется только для Билшок.

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\Http_module.txt
--------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
#ocs_principle_of_operation
#ocs_mop
#lab_principle_of_operation
#http_principle_of_operation
------------------------------
Пример МОР по перенаправлению IMSI в Http Module с OCS prod в OCS Lab.
Http Module отвечает за CS, соответственно, перенаправляются запросы, касающиеся голоса и СМС.

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\Http_Module_in_lab_testing\readme.txt
----------------------------------------------------------------------------------------------------
#ocs_principle_of_operation
#ocs_mop
#http_principle_of_operation
------------------------------
Информация по тестированию data в Lab OCS и по использованию http relay в Lab.

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS

### Search in ALARMS by folder names

In [15]:
import os

paths = [
        'c:/W_DATA_ROAM/ALARMS/',
        ]

for path in paths:
    names = os.listdir(path)
    [print(normpath(path + name)) for name in names if name.upper().find('GTMTG')!=-1]

c:\W_DATA_ROAM\ALARMS\MOC_PARTNET_GTMTG_#2045114_181120
c:\W_DATA_ROAM\ALARMS\PS_BRATC_COLTM_CRITC_GTMTG_Telefonica_Central_America_181019


### Search in Allowed Lists and others

In [12]:
downloads = 'C:/Users/balob/Downloads'

#Like expression for Allowed List
df_al=pd.read_csv(join(downloads,'DMI.dmi_allowed_list_export_Tue_Feb_26_2019.csv'))
#df_al_lab=pd.read_csv('LAB_DMI.dmi_allowed_list_export_Wed_Oct_03_2018.csv')

#df_dmi_diam_oper=pd.read_csv(join(download,''))
df_np=pd.read_csv(join(downloads,'DMI.dmi_netpfx_export_Wed_Feb_13_2019.csv'))
#df_did=pd.read_csv('HRR.hrr_did_export_Wed_Aug_08_2018.csv')
#f_tap_ocs=pd.read_csv('tap_ocs_analysis_180817.csv')

In [21]:
#df_al[(df_al['PLMN code'].str.startswith('MKDCC', na=False)) & (df_al['Sponsor']=='')]
df_al.loc[(df_al['PLMN code'].str.startswith('ITAOM', na=False)) &
          (df_al['Sponsor'].str.contains('STI', na=False)), ['Sponsor','PLMN code']].sort_values(by=['Sponsor','PLMN code'])

,Sponsor,PLMN code
1040,Partner_STI_Comb,ITAOM


In [12]:
str(df_np[df_np.Operator == 'TURTK'].Prefix.unique().tolist())
df_np[(df_np.Operator.str.startswith('MOZVT',na=False))] #.to_csv(join(downloads,'Prefix.csv')) # & (df_np.SSN==7)

,Operator,Prefix,Number/Range,SSN,MCC,MNC
1190,MOZVT,25886000,1,7,643,3
1191,MOZVT,25886000,1,149,643,3
5425,MOZVT,258860001201,0,1,643,3
5426,MOZVT,84970025886,1,7,643,3
5427,MOZVT,84970025886,1,149,643,3


In [65]:
plmn = 'ESPVV'
df1 = df_al.loc[(df_al['PLMN code'].str.startswith(plmn, na=False)) &
          (df_al['Sponsor'].str.contains('', na=False)), ['Sponsor','PLMN code']].sort_values(by=['Sponsor','PLMN code'])
df2 = df_np[(df_np.Operator.str.startswith(plmn,na=False))] #.to_csv(join(downloads,'Prefix.csv')) # & (df_np.SSN==7)
display(df1)
display(df2)

,Sponsor,PLMN code
3248,P4_telzar,ESPVV
3190,PA_Telzar_Soft,ESPVV
179,Partner,ESPVV
2707,Partner_All,ESPVV
2816,Partner_Dialoq_A,ESPVV
3128,Partner_RedT_Sof,ESPVV
1452,Partner_RedTea,ESPVV
1917,Partner_WeStream,ESPVV
1346,Partner_mon_euro,ESPVV
537,Partner_mondic,ESPVV


,Operator,Prefix,Number/Range,SSN,MCC,MNC
14,ESPVV,34607,1,7,214,1
15,ESPVV,34607,1,149,214,1
16,ESPVV,34697,1,7,214,1
17,ESPVV,34697,1,149,214,1
5764,ESPVV,3469700010,0,7,214,1
5765,ESPVV,3469700089,0,7,214,1
9245,ESPVV,3469700410,0,149,214,1
9246,ESPVV,3469700489,0,149,214,1
10165,ESPVV,34697004100,0,1,214,1


In [215]:
str(df_np[(df_np.Prefix.str.startswith('6683',na=False))].Prefix.unique().tolist())
df_np[(df_np.Prefix.str.startswith('51955',na=False))]

,Operator,Prefix,Number/Range,SSN,MCC,MNC
12787,PERN3,5195539,1,7,716,17
12789,PERN3,51955392201,0,7,716,17
12790,PERN3,51955392202,0,7,716,17
12794,PERN3,51955392203,0,149,716,17
12795,PERN3,51955392203,0,7,716,17
12796,PERN3,51955392204,0,7,716,17
12797,PERN3,51955392204,0,149,716,17
12798,PERN3,51955399136,0,149,716,17
12799,PERN3,51955399136,0,7,716,17
12800,PERN3,51955399165,0,7,716,17


### Hex to Dec conversion

In [227]:
hex_str = 'aa'
dec_str = '80'
dec_int = 80
bin_str = '1010'

# Hex to dec
print('Hex to dec')
print(int(hex_str,16))

# Dec to hex
print('Dec to hex')
print(hex(dec_int))
print('%x' % dec_int)

# Dec to bin
print('Dec to bin')
print(bin(dec_int)[2:].zfill(8))

# Bin to dec
print('Bin to dec')
print(int(bin_str,2))

# Hex to bin
print('Hex to bin')
print(bin(int(hex_str, 16))[2:].zfill(8))
print(bin(0xABC123EFFF))
f'{0xAB:0>16b}'

Hex to dec
170
Dec to hex
0x50
50
Dec to bin
01010000
Bin to dec
10
Hex to bin
10101010
1010101111000001001000111110111111111111


'0000000010101011'

In [188]:
f'{bin(2)} - {bin(61)} - {bin(6)}'
#2.61.6

'0b10 - 0b111101 - 0b110'

In [186]:
print(bin(10915))
a = '10101010100011'
# = '00000011001101'
print(int(a[-14:-11],2),'-',int(a[-11:-3],2),'-',int(a[-3:],2) )

0b10101010100011
5 - 84 - 3


### Rename files

In [ ]:
import os
import re
path = 'c:/Users/balob/Downloads/new1/'
prefix = 'PYCON'
topic = ''
os.chdir(path)
names = sorted(filter(os.path.isfile, os.listdir('.')), key=os.path.getmtime)
for i,name in enumerate(names):
    short_name = re.sub('[!#?«»,() \+\-؟]','',name[:-4:][:50])
    if i < 10:
        number = '0'+str(i)
    else:
        number = str(i)
    new_name = '{}_{}_{}.mp4'.format(prefix,number,short_name)
    print('{} -> {}'.format(name,new_name))
    os.rename(name,new_name)

### Чтение логов с удаленного сервера SFTP (stat_GrayLog.log)

In [86]:
import paramiko
print_lines = 50
host = '172.19.11.191'
user = 'pavel'
secret = 'RzAZPjMjvujCPjU88bpevQp'
file_path_name = '/home/tracer/LOG/stat_GrayLog.log'
port = 22
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname=host, username=user, password=secret, port=port)
sftp_client = client.open_sftp()
remote_file = sftp_client.open(file_path_name)
try:
    #for line in remote_file:
    for i,line in enumerate(reversed(list(remote_file))):
        if i < print_lines:
            if 'Total' in line:
                print(i,line,end='')
            #print(i,line,end='')
        else:
            break
finally:
    remote_file.close()
client.close()
# (https://stackoverflow.com/questions/1596963/read-a-file-from-server-with-ssh-using-python)

0 2019-01-24 05:00:45,431 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:43.978321
8 2019-01-23 05:00:47,552 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:45.941926
16 2019-01-22 05:00:49,749 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:45.060281
24 2019-01-21 05:00:49,224 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:44.623832
32 2019-01-20 05:00:47,066 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:45.239989
40 2019-01-19 05:00:45,864 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:43.761820
48 2019-01-18 05:00:46,195 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:43.864928


### Pricing counting

In [192]:
moc=1.5
mtc=0
text=0.25
data=7.37

tariff=[moc,mtc,text,data]

print("Price P4")
for k,v in {'1.2':1.2,'1.2*1.5':1.2*1.5,'1.2*1.5*1.15':1.2*1.5*1.15,'1.2*1.5*1.30':1.2*1.5*1.30,'2':1.2*2}.items():
    print("Price {:<15}:".format(k),[round(i*v,6) for i in tariff])
print("\nPrice Partner")
for k,v in {'1.5':1.5,'1.5*1.15':1.5*1.15,'1.5*1.30':1.5*1.30,'2':2}.items():
    print("Price {:<15}:".format(k),[round(i*v,6) for i in tariff])

Price P4
Price 1.2            : [1.8, 0.0, 0.3, 8.844]
Price 1.2*1.5        : [2.7, 0.0, 0.45, 13.266]
Price 1.2*1.5*1.15   : [3.105, 0.0, 0.5175, 15.2559]
Price 1.2*1.5*1.30   : [3.51, 0.0, 0.585, 17.2458]
Price 2              : [3.6, 0.0, 0.6, 17.688]

Price Partner
Price 1.5            : [2.25, 0.0, 0.375, 11.055]
Price 1.5*1.15       : [2.5875, 0.0, 0.43125, 12.71325]
Price 1.5*1.30       : [2.925, 0.0, 0.4875, 14.3715]
Price 2              : [3.0, 0, 0.5, 14.74]


## Testing

In [317]:
from os.path import join
import re
import xml.etree.ElementTree as et

downloads = 'C:/Users/balob/Downloads'
file_ir21 = 'IR21_INDAT_Bharti Airtel Limited_20180307_03.xml'

tree = et.parse(join(downloads, file_ir21))
root = tree.getroot()
print(root.tag)
ns = re.match(r'{.*}', root.tag).group(0)
print(ns)

print(f'Root :\n   tag: {root.tag}\n   attrib: {root.attrib},\n   text: {root.text}')
for neighbor in root.iter(f'{ns}OrganisationName'):
    print(f'Neighbor :\n   tag: {neighbor.tag}\n   attrib: {neighbor.attrib},\n   text: {neighbor.text}')

{https://infocentre.gsm.org/TADIG-RAEX-IR21}TADIGRAEXIR21
{https://infocentre.gsm.org/TADIG-RAEX-IR21}
Root :
   tag: {https://infocentre.gsm.org/TADIG-RAEX-IR21}TADIGRAEXIR21
   attrib: {'{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'https://infocentre.gsm.org/TADIG-RAEX-IR21 tadig-raex-ir21-10.1.xsd'},
   text: 
  
Neighbor :
   tag: {https://infocentre.gsm.org/TADIG-RAEX-IR21}OrganisationName
   attrib: {},
   text: Bharti Airtel Limited


In [275]:
from bs4 import BeautifulSoup
from os.path import join
import pandas as pd
from pandas import Series, DataFrame

downloads = 'C:/Users/balob/Downloads'
file_ir21 = 'IR21_INDAT_Bharti Airtel Limited_20180307_03.xml'

with open(join(downloads,file_ir21)) as inf:
    ir21 = inf.read()

xml_ir21 = BeautifulSoup(ir21,'xml')

msisdn_list = []
for MSISDN_NumberRanges in xml_ir21.findAll('MSISDN_NumberRanges'):
    for MSISDN_RangeData in MSISDN_NumberRanges.findAll('MSISDN_RangeData'):
        for NumberRange in  MSISDN_RangeData.findAll('NumberRange'):
            msisdn_list.append([NumberRange.CC.text + NumberRange.NDC.text, 'msisdn'])

nn_list = []
for MSISDN_NumberRanges in xml_ir21.findAll('GT_NumberRanges'):
    for MSISDN_RangeData in MSISDN_NumberRanges.findAll('RangeData'):
        for NumberRange in  MSISDN_RangeData.findAll('NumberRange'):
            nn_list.append([NumberRange.CC.text + NumberRange.NDC.text, 'nn'])
            
msrn_list = []
for MSISDN_NumberRanges in xml_ir21.findAll('MSRN_NumberRanges'):
    for NumberRange in  MSISDN_NumberRanges.findAll('NumberRange'):
        msrn_list.append([NumberRange.CC.text + NumberRange.NDC.text, 'msrn'])
        
e164_list = []
e164_list.extend(msisdn_list)
e164_list.extend(nn_list)
e164_list.extend(msrn_list)

for tadigs in xml_ir21.findAll('SenderTADIG'):
    tadig = tadigs.text
    print(tadig)
    
df = DataFrame(e164_list, columns = ['e164', 'type'])
df['tadig'] = tadig
df['tadig'] = 'IRLDF'

display(df)

pd.merge(df_np.loc[df_np.Operator == tadig, ['Operator', 'Prefix', 'Number/Range', 'SSN']],
         df[df.type == 'nn'], how='left', left_on=['Operator', 'Prefix'], right_on=['tadig', 'e164'])

INDAT


,e164,type,tadig
0,917033,msisdn,IRLDF
1,917042,msisdn,IRLDF
2,917051,msisdn,IRLDF
3,917054,msisdn,IRLDF
4,917063,msisdn,IRLDF
5,917070,msisdn,IRLDF
6,917077,msisdn,IRLDF
7,917191,msisdn,IRLDF
8,917192,msisdn,IRLDF
9,917194,msisdn,IRLDF


NameError: name 'df_np' is not defined

In [40]:
df_np.head()

,Operator,Prefix,Number/Range,SSN,MCC,MNC
0,GRCCO,3097,1,7,202,2
1,GRCCO,3097,1,149,202,2
2,GRCSH,30699,1,7,202,10
3,GRCSH,30699,1,149,202,10
4,CHEOR,4178931,1,7,228,3


In [220]:
dmi_subs_file_name = 'export_dmi_subs.csv'
df_dmi = pd.read_csv(join(downloads, dmi_subs_file_name), dtype='str')
df_dmi.head(3)

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME
0,Cellact,425180020000000,972557012345,425019620010108,1,Partner_Combined
1,Cellact,425180020000000,972557012345,260060140068752,2,P4_Combined
2,Cellact,425180020000002,NaN,425019620011335,1,Partner_Combined


In [234]:
sql_srt='''
SELECT si.IMSI
,mi.IMSI AS S_IMSI
,ss.STATUS
FROM SUBSCRIBER_IMSIS si
LEFT JOIN MULTI_IMSI mi ON si.subscriber_id = mi.subscriber_id
LEFT JOIN SUBSCRIBER_STATUS ss ON si.subscriber_id = ss.subscriber_id
WHERE ss.STATUS = 'Active' AND ss.END_DATE IS NULL
'''
with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn: # prod connection
#with rb.MssqlConnect('172.20.39.14', '10028', 'ocslab', 'P9fznRUzyDSNO43j') as cnxn: # lab connection
    df_ocs = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
df_ocs.head(3)

,IMSI,S_IMSI,STATUS
0,724540540145413,425019620000005,Active
1,724540540197011,425019620000016,Active
2,724540540152352,425019620000003,Active


In [235]:
df_ocs.count()

IMSI      1356753
S_IMSI    1356750
STATUS    1356753
dtype: int64

In [236]:
df_ocs_dmi = pd.merge(df_dmi, df_ocs, how='outer', on=['IMSI', 'S_IMSI'])
df_ocs_dmi.head(3)

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,STATUS
0,Cellact,425180020000000,972557012345,425019620010108,1,Partner_Combined,Active
1,Cellact,425180020000000,972557012345,260060140068752,2,P4_Combined,Active
2,Cellact,425180020000002,NaN,425019620011335,1,Partner_Combined,NaN


In [239]:
df_ocs_dmi[df_ocs_dmi.MVNO_NAME.isnull()].count()

MVNO_NAME       0
IMSI         2870
MSISDN          0
S_IMSI       2867
PROF_ID         0
SP_NAME         0
STATUS       2870
dtype: int64

In [238]:
df_ocs_dmi[df_ocs_dmi.MVNO_NAME.isnull()].head()

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,STATUS
1378092,NaN,425019612599996,NaN,425019612599996,NaN,NaN,Active
1378093,NaN,425019612599995,NaN,425019612599995,NaN,NaN,Active
1378094,NaN,99999425019620000100,NaN,425019620000100,NaN,NaN,Active
1378095,NaN,99999425019620000101,NaN,425019620000101,NaN,NaN,Active
1378096,NaN,99999425019620000103,NaN,425019620000103,NaN,NaN,Active


In [243]:
df_ocs_dmi[(df_ocs_dmi.IMSI.notnull()) & (df_ocs_dmi.S_IMSI.isnull())].head()

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,STATUS
1380932,NaN,724540540268812,NaN,None,NaN,NaN,Active
1380942,NaN,240400000030323,NaN,None,NaN,NaN,Active
1380947,NaN,724540540339447,NaN,None,NaN,NaN,Active


In [253]:
imsi = '240400000030323'
display(df_dmi[df_dmi.IMSI == imsi])
display(df_ocs[df_ocs.IMSI == imsi])
display(df_ocs_dmi[df_ocs_dmi.IMSI == imsi])

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME
38684,Rewicom,240400000030323,46722147950,425019620005915,1,Partner
38685,Rewicom,240400000030323,46722147950,260060140007566,2,P4


,IMSI,S_IMSI,STATUS
895962,240400000030323,None,Active


,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,STATUS
38684,Rewicom,240400000030323,46722147950,425019620005915,1,Partner,NaN
38685,Rewicom,240400000030323,46722147950,260060140007566,2,P4,NaN
1380942,NaN,240400000030323,NaN,None,NaN,NaN,Active


In [100]:
class Employee():
    def __init__(self, first, last, salary):
        self.first = first
        self.last = last
        self.salary = salary
    @classmethod
    def from_string(cls, emp_str):
        first, last, salary = emp_str.split('-')
        return cls(first, last, salary)
emp_str_1 = 'Joe-Doe-70000'
emp_1 = Employee.from_string(emp_str_1)

In [136]:
class Developer(Employee):
    def __init__(self, first, last, salary, prog_lang):
        super().__init__(first, last, salary)
        # или
        #Employee.__init__(self, first, last, salary)
        self.prog_lang = prog_lang
    @classmethod
    def from_string(cls, dev_str):
        first, last, salary, prog_lang = dev_str.split('-')
        return cls(first, last, salary, prog_lang)
dev_str_1 = 'Joe-Doe-70000-Python'
dev_1 = Developer.from_string(dev_str_1)

In [148]:
isinstance(dev_1, Employee) # True
isinstance(emp_1, Employee) # True
isinstance(dev_1, Developer) # True
isinstance(emp_1, Developer) # False
issubclass(Developer, Employee) # True
issubclass(Employee, Developer) # False

False

In [6]:
37.83 * 67

2534.6099999999997

In [257]:
%reset -f